In [2]:

import sleap_io

from concurrent.futures import ThreadPoolExecutor
import tqdm

from pathlib import Path

import subprocess
import os

In [75]:
Test_labels = sleap_io.load_slp("/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated_tracked_ball.slp")

In [76]:
Test_labels

Labels(labeled_frames=261006, videos=1, skeletons=1, tracks=0, suggestions=0)

In [5]:
# Frame 1

Test_labels[0].instances

[PredictedInstance(points=[[46.98500442504883, 287.9368591308594]], track=None, score=0.51, tracking_score=0.00),
 PredictedInstance(points=[[239.2644805908203, 439.47406005859375]], track=None, score=0.42, tracking_score=0.00)]

In [9]:
Test_labels[0].instances[0].track = "track_1"

In [10]:
Test_labels[0].instances[0].track

'track_1'

In [40]:
next(iter(Test_labels[0].instances[0].points.items()))[1].x

46.98500442504883

In [49]:
Test_labels[0].instances[0].points


{Node(name='centre'): PredictedPoint(x=46.98500442504883, y=287.9368591308594, visible=True, complete=False, score=0.7572789192199707)}

In [77]:
Test_labels.tracks.append(sleap_io.Track(name="track_1"))
Test_labels.tracks.append(sleap_io.Track(name="track_2"))

In [78]:
Subset_Test_labels = Test_labels[0:10]

In [80]:
Subset_Test_labels

[LabeledFrame(video=Video(filename="/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated.mp4", shape=(261006, 570, 260, 1), backend=MediaVideo), frame_idx=0, instances=[PredictedInstance(points=[[46.98500442504883, 287.9368591308594]], track=None, score=0.51, tracking_score=0.00), PredictedInstance(points=[[239.2644805908203, 439.47406005859375]], track=None, score=0.42, tracking_score=0.00)]),
 LabeledFrame(video=Video(filename="/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated.mp4", shape=(261006, 570, 260, 1), backend=MediaVideo), frame_idx=1, instances=[PredictedInstance(points=[[46.986419677734375, 287.9563903808594]], track=None, score=0.51, tracking_score=0.00), PredictedInstance(points=[[239.25283813476562, 439.4856872558594]], track=None, score=0.43, tracking_score=0.00)]),
 LabeledFrame(video=Video(filename="/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_V

In [91]:
Test_labels.tracks

[Track(name='track_1'), Track(name='track_2')]

In [3]:
def prune_instances_tracks(input, verbose=False):
    # Load the SLP file
    print("Loading labels")
    
    input_path = Path(input)
    
    # Check if the file has already been processed
    if "_processed" in input_path.stem:
        print("File already processed, skipping")
        return
    
    Labels = sleap_io.load_slp(input)
    
    if len(Labels.tracks) > 2:
        print("Too many tracks defined, removing all")
        for frame in Labels:
            for instance in frame.instances:
                instance.track = None
        Labels.tracks = []
    
    if len(Labels.tracks) == 0:
        Labels.tracks.append(sleap_io.Track(name="track_1"))
        Labels.tracks.append(sleap_io.Track(name="track_2"))
    
    for frame in Labels:
        if verbose:
            print(f"Processing frame {frame.frame_idx}")
        
        # Sort by confidence
        frame.instances.sort(key=lambda x: x.score, reverse=True)
        if verbose:
            print(f"Instances sorted by confidence: {frame.instances}")
        
        # Keep the 2 highest confidence that are not too close in x
        kept_instances = []
        for instance in frame.instances:
            if len(kept_instances) == 0:
                kept_instances.append(instance)
                x1 = next(iter(kept_instances[0].points.items()))[1].x
                if verbose:
                    print(f"First kept instance x: {x1}")
            elif len(kept_instances) == 1:
                first_x = next(iter(kept_instances[0].points.items()))[1].x
                inst_x = next(iter(instance.points.items()))[1].x
                if verbose:
                    print(f"Comparing x values: first_x={first_x}, inst_x={inst_x}")
                
                if abs(first_x - inst_x) > 90:
                    kept_instances.append(instance)
                    x2 = next(iter(kept_instances[1].points.items()))[1].x
                    if verbose:
                        print(f"Second kept instance x: {x2}")
            else:
                break
        
        # Sort by x
        kept_instances.sort(key=lambda x: next(iter(x.points.items()))[1].x)
        if verbose:
            print(f"Kept instances sorted by x: {kept_instances}")
        
        # Update the frame instances
        frame.instances = kept_instances
        if verbose:
            print(f"Updated frame instances: {frame.instances}")
        
        # Ensure track_1 is always left (lower x value) and track_2 is always right (higher x value)
        frame.instances.sort(key=lambda x: next(iter(x.points.items()))[1].x)
        for n, instance in enumerate(frame.instances):
            if verbose:
                print(f"Frame {frame.frame_idx}, instance {n}")
            instance_x = next(iter(instance.points.items()))[1].x
            if verbose:
                print(f"Instance x: {instance_x}")
            # Add track names
            instance.track = Labels.tracks[n]
            if verbose:
                print(f"Assigned track name: {instance.track}")
        
        if verbose:
            print("Updated frame instances")
            print(frame.instances)
                
    print("Saving updated labels")
    sleap_io.save_slp(Labels, input)
    
    # # Rename the file to indicate it has been processed
    processed_filename = input_path.stem + "_processed" + input_path.suffix
    processed_path = input_path.with_name(processed_filename)
    try:
        os.rename(input, processed_path)
        print(f"Renamed file to {processed_path}")
    except OSError as e:
        print(f"Error renaming file: {e}")


In [84]:
Test_labels

Labels(labeled_frames=261006, videos=1, skeletons=1, tracks=2, suggestions=0)

In [86]:
Test_labels[145].instances

[PredictedInstance(points=[[46.944915771484375, 287.9378356933594]], track="track_1", score=0.50, tracking_score=0.00),
 PredictedInstance(points=[[239.24050903320312, 439.6255187988281]], track="track_2", score=0.42, tracking_score=0.00)]

In [83]:
Test_labels

Test_labels.tracks

[Track(name='track_1'), Track(name='track_2')]

In [88]:
# Save the updated labels
sleap_io.save_slp(filename="/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated_tracked_ball.slp", labels=Test_labels)

In [13]:
def reassign_tracks_based_on_x(input, verbose=False):
    # Load the SLP file
    print("Loading labels")
    
    Labels = sleap_io.load_slp(input)
    
    if len(Labels.tracks) != 2:
        print("There should be exactly 2 tracks defined")
        return
    
    for frame in tqdm.tqdm(Labels, desc="Reassigning tracks"):
        if verbose:
            print(f"Processing frame {frame.frame_idx}")
        
        # Sort instances by x value
        frame.instances.sort(key=lambda x: next(iter(x.points.items()))[1].x)
        
        for n, instance in enumerate(frame.instances):
            if verbose:
                print(f"Frame {frame.frame_idx}, instance {n}")
            instance_x = next(iter(instance.points.items()))[1].x
            if verbose:
                print(f"Instance x: {instance_x}")
            # Reassign track names
            instance.track = Labels.tracks[n]
            if verbose:
                print(f"Reassigned track name: {instance.track}")
        
        if verbose:
            print("Updated frame instances")
            print(frame.instances)
                
    print("Saving updated labels")
    sleap_io.save_slp(Labels, input)

In [97]:
prune_instances_tracks("/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena4/Right/arena4_right_rotated_tracked_ball.slp", verbose=False)

Too many tracks defined, removing all


Processing frames: 100%|██████████| 261006/261006 [00:01<00:00, 182425.31it/s]


Saving updated labels


In [99]:
reassign_tracks_based_on_x("/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena4/Right/arena4_right_rotated_tracked_ball.slp", verbose=False)

Loading labels


Reassigning tracks: 100%|██████████| 261006/261006 [00:01<00:00, 145025.56it/s]


Saving updated labels


In [16]:
def process_dataset_in_parallel(datasets, verbose=False):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(prune_instances_tracks, dataset, verbose) for dataset in datasets]
        for future in tqdm.tqdm(futures, desc="Processing datasets"):
            future.result()

In [5]:
Data_dir = Path("/mnt/upramdya_data/MD/F1_Tracks/Videos/")

# Find all SLP files that have "ball" in their name in the data directory

SLP_files = list(Data_dir.rglob("*ball*.slp"))

# Prune out the processed files
SLP_files = [file for file in SLP_files if "_processed" not in file.stem]

SLP_files

[PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena6/Left/arena6_left_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena5/Right/arena5_right_rotated_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena5/Left/arena5_left_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena8/Right/arena8_right_rotated_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena8/Left/arena8_left_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena3/Right/arena3_right_rotated_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena3/Left/arena3_left_tracked_ball.slp'),
 PosixPath('/mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_en

In [6]:
for slp in SLP_files:

    prune_instances_tracks(slp, verbose=False)

Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena6/Left/arena6_left_tracked_ball_processed.slp
Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena5/Right/arena5_right_rotated_tracked_ball_processed.slp
Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena5/Left/arena5_left_tracked_ball_processed.slp
Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena8/Right/arena8_right_rotated_tracked_ball_processed.slp
Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240926_F1_3mm_ends_Videos_Checked/arena8/Left/arena8_left_tracked_ball_processed.slp
Loading labels
Saving updated labels
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240

In [19]:
# Rename all processed file to add "_processed" to the filename

for slp_file in SLP_files:
    slp_file = Path(slp_file)  # Ensure slp_file is a Path object
    processed_filename = slp_file.stem + "_processed" + slp_file.suffix
    processed_path = slp_file.with_name(processed_filename)
    os.rename(slp_file, processed_path)
    print(f"Renamed file to {processed_path}")

Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Left/arena2_left_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena7/Right/arena7_right_rotated_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena7/Left/arena7_left_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena6/Right/arena6_right_rotated_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena6/Left/arena6_left_tracked_ball_processed.slp
Renamed file to /mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena5/Right/arena5_right_rotated_tracked

In [ ]:
# remove the "_processed" from the filename

In [7]:
# Finally we need to call sleap-convert to convert the newly edited slp files to h5
def convert_slp_to_h5(input, conda_env="sleap"):
    # Call a subprocess to run sleap-convert within the specified Conda environment
    
    # Convert the .slp file to .h5 format for analysis
    sleap_convert_cmd = f"""
    source $(conda info --base)/etc/profile.d/conda.sh && \
    conda activate {conda_env} && \
    sleap-convert "{input}" --format analysis
    """
    
    # Print the command for debugging
    print("Running command:", sleap_convert_cmd)
    
    # Check if the input file exists
    if not os.path.exists(input):
        print(f"Input file does not exist: {input}")
        return
    
    # Run the command
    try:
        subprocess.run(sleap_convert_cmd, shell=True, check=True, executable='/bin/bash')
        print("Conversion to .h5 format complete.")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")
    except OSError as e:
        print(f"OS error: {e}")



In [8]:
[convert_slp_to_h5(file) for file in SLP_files]

Running command: 
    source $(conda info --base)/etc/profile.d/conda.sh &&     conda activate sleap &&     sleap-convert "/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated_tracked_ball.slp" --format analysis
    


Traceback (most recent call last):
  File "/home/durrieu/miniforge3/envs/sleap/bin/sleap-convert", line 33, in <module>
    sys.exit(load_entry_point('sleap==1.3.4', 'console_scripts', 'sleap-convert')())
  File "/home/durrieu/miniforge3/envs/sleap/lib/python3.7/site-packages/sleap/io/convert.py", line 170, in main
    video=video,
  File "/home/durrieu/miniforge3/envs/sleap/lib/python3.7/site-packages/sleap/info/write_tracking_h5.py", line 440, in main
    write_occupancy_file(output_path, data_dict, transpose=True)
  File "/home/durrieu/miniforge3/envs/sleap/lib/python3.7/site-packages/sleap/info/write_tracking_h5.py", line 261, in write_occupancy_file
    with h5.File(output_path, "w") as f:
  File "/home/durrieu/miniforge3/envs/sleap/lib/python3.7/site-packages/h5py/_hl/files.py", line 427, in __init__
    swmr=swmr)
  File "/home/durrieu/miniforge3/envs/sleap/lib/python3.7/site-packages/h5py/_hl/files.py", line 196, in make_fid
    fid = h5f.create(name, h5f.ACC_TRUNC, fapl=fapl, 

Error during conversion: Command '
    source $(conda info --base)/etc/profile.d/conda.sh &&     conda activate sleap &&     sleap-convert "/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Right/arena2_right_rotated_tracked_ball.slp" --format analysis
    ' returned non-zero exit status 1.
Running command: 
    source $(conda info --base)/etc/profile.d/conda.sh &&     conda activate sleap &&     sleap-convert "/mnt/upramdya_data/MD/F1_Tracks/Videos/240924_F1_3mm_ends_Videos_Checked/arena2/Left/arena2_left_tracked_ball.slp" --format analysis
    


KeyboardInterrupt: 